In [1]:
%%capture
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
from skimage import io
import imutils
from skimage.feature import local_binary_pattern
from IPython.display import clear_output
#from google.colab import drive
import os
import glob
#from tqdm import tqdm
from tqdm.notebook import tqdm
#drive.mount('/content/drive')
%matplotlib inline

In [2]:
# if not os.path.exists('/content/drive/MyDrive/ICDAR2013'):
#     os.system('cp /content/drive/MyDrive/ICDAR2013.zip ./')
#     os.system('unzip ICDAR2013.zip')
# if not os.path.exists('/content/drive/MyDrive/CMP2023'):
#     os.system('cp /content/drive/MyDrive/CMP2023.zip ./')
#     os.system('unzip CMP2023.zip')

In [3]:
def remove_shadow(img):
    rgb_planes = cv2.split(img)
    kernel = np.ones((20, 20), np.float32)/400
    result_planes = []
    for plane in rgb_planes:
        f = cv2.filter2D(plane, -1, kernel)
        result_planes.append(f)
    result_planes = np.array(result_planes, np.int32)
    result_planes = (rgb_planes-result_planes)
    result = cv2.merge(result_planes)
    result[result < 0] = 0
    result = result*9
    result = np.array(result, np.uint8)
    return result

def show_img(imgs, size=10):
    if type(imgs) is not list:
        imgs = [imgs]
    fig = plt.figure(figsize=(size, size))
    for i in range(len(imgs)):
        a = fig.add_subplot(1, len(imgs), i+1)
        plt.imshow(imgs[i], cmap='gray')
        plt.axis("off")
    plt.show()

def save_img(imgs, name='img', location='./', size=0):
    if type(imgs) is not list:
         if (size != 0):
            imgs = imutils.resize(imgs, width=size)
         io.imsave(location+'/'+name+'.png',imgs)
    else:
        if (size != 0):
            for i in tqdm(range(len(imgs))):
                imgs[i] = imutils.resize(imgs[i], width=size)
        for i in tqdm(range(len(imgs))):
            io.imsave(location+'/'+name+str(i)+'.png',imgs[i])

def read_img(path,size=0):
    img = io.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if size!=0:
        img = imutils.resize(img, width=size)
    img = img[:int(img.shape[0]*(1-0.1))]
    img = img[:, int(img.shape[1]*0.05):]
    img = img[:, :int(img.shape[1]*(1-0.05))]
    img = img[int(img.shape[0]*0.05):]
    return img

def clip_writing_area(img):
    shadow_free_img = remove_shadow(img)
    binary = (shadow_free_img > 90).astype(np.uint8)
    kernel = np.ones((3, 3), np.uint8)
    erosion = cv2.erode(binary, kernel, iterations=2)
    dilate = cv2.dilate(erosion, kernel, iterations=30)
    coords = cv2.findNonZero(dilate)
    x, y, w, h = cv2.boundingRect(coords)
    rect = img[y:y+h, x:x+w]
    return rect

def lbp(img):
    lbp = local_binary_pattern(img, 8, 1, method="uniform")
    return lbp

def histogram(img):
    hist,bins = np.histogram(img.ravel(),256,[0,256])
    return hist

In [4]:
#!mkdir -p "./Datasets/CMP2023/Female/"
#!mkdir -p "./Datasets/CMP2023/Male/"

In [9]:
index=1
for dirname, _, filenames in os.walk('./CMP2023/Female/'):
    for filename in tqdm(filenames):
        filepath = dirname+filename
        try:
            img = read_img(filepath,3000)
            img = clip_writing_area(img)
            save_img(img,"F"+str(index),"./Datasets/CMP2023_Clipped/Female/")
            index+=1
        except:
            print(filepath)

index=1
for dirname, _, filenames in os.walk('./CMP2023/Male/'):
    for filename in tqdm(filenames):
        filepath = dirname+filename
        try:
            img = read_img(filepath,3000)
            img = clip_writing_area(img)
            save_img(img,"M"+str(index),"./Datasets/CMP2023_Clipped/Male/")
            index+=1
        except:
            print(filepath)

  0%|          | 0/132 [00:00<?, ?it/s]

./CMP2023/Female/.DS_Store


  0%|          | 0/233 [00:00<?, ?it/s]

./CMP2023/Male/.DS_Store


In [6]:
#!mkdir -p "./ICDAR13_Clipped/Female/"
#!mkdir -p "./ICDAR13_Clipped/Male/"

In [7]:
index=1
for dirname, _, filenames in os.walk('./ICDAR2013/Female/'):
    for filename in tqdm(filenames):
        filepath = dirname+filename
        try:
            img = read_img(filepath,3000)
            img = clip_writing_area(img)
            save_img(img,"F"+str(index),"./Datasets/ICDAR2013_Clipped/Female/")
            index+=1
        except:
            print(filepath)

index=1
for dirname, _, filenames in os.walk('./ICDAR2013/Male/'):
    for filename in tqdm(filenames):
        filepath = dirname+filename
        try:
            img = read_img(filepath,3000)
            img = clip_writing_area(img)
            save_img(img,"M"+str(index),"./Datasets/ICDAR2013_Clipped/Male/")
            index+=1
        except:
            print(filepath)

  0%|          | 0/564 [00:00<?, ?it/s]

  0%|          | 0/286 [00:00<?, ?it/s]

In [8]:
import os
res = os.listdir("./ICDAR2013/Male/")
print(len(res))

res = os.listdir("./ICDAR2013_Clipped/Male/")
print(len(res))

res = os.listdir("./ICDAR2013/Female/")
print(len(res))

res = os.listdir("./ICDAR2013_Clipped/Female/")
print(len(res))

res = os.listdir("./CMP2023/Male/")
print(len(res))

res = os.listdir("./CMP2023_Clipped/Male/")
print(len(res))

res = os.listdir("./CMP2023/Female/")
print(len(res))


res = os.listdir("./CMP2023_Clipped/Female/")
print(len(res))


286


FileNotFoundError: [Errno 2] No such file or directory: './ICDAR2013_Clipped/Male/'

In [ ]:
#!mkdir -p "/content/drive/MyDrive/tempNN"
#!cp -r "./CMP2023" "/content/drive/MyDrive/tempNN/"
#!cp -r "./CMP2023_Clipped" "/content/drive/MyDrive/tempNN/"
#!cp -r "./ICDAR2013" "/content/drive/MyDrive/tempNN/"
#!cp -r "./ICDAR2013_Clipped" "/content/drive/MyDrive/tempNN/"

: 

: 